# Running Biomechanics Data Set

Reginaldo K Fukuchi, Jan 2023, reginaldo.fukuchi@ufabc.edu.br

This NB imports c3d files and convert data into ASCII files for markers trajectories and forces separately.

In [1]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

We're running Colab


In [2]:
if IN_COLAB:
  # Mount the Google Drive at mount
  mount='/content/gdrive'
  print("Colab: mounting Google drive on ", mount)

  drive.mount(mount)

  # Switch to the directory on the Google Drive that you want to use
  import os
  drive_root = mount + "/MyDrive/Github/UFABC_UofC_datasets"
  
  # Create drive_root if it doesn't exist
  create_drive_root = False
  if create_drive_root:
    print("\nColab: making sure ", drive_root, " exists.")
    os.makedirs(drive_root, exist_ok=True)
  
  # Change to the directory
  print("\nColab: Changing directory to ", drive_root)
  %cd $drive_root

Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive

Colab: Changing directory to  /content/gdrive/MyDrive/Github/UFABC_UofC_datasets
/content/gdrive/MyDrive/Github/UFABC_UofC_datasets


In [3]:
!git init

Reinitialized existing Git repository in /content/gdrive/MyDrive/Github/UFABC_UofC_datasets/.git/


In [4]:
# Prepare environment
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook

In [5]:
# Import data
pathname = os.path.join(drive_root,'data')

## Import C3D RBDS file using EZC3D
https://github.com/pyomeca/ezc3d

In [7]:
fc_path = os.path.join(drive_root,'functions')
fc_path

'/content/gdrive/MyDrive/Github/UFABC_UofC_datasets/functions'

In [ ]:
import sys
sys.path.insert(1, fc_path)
from svdt import svdt

In [ ]:
!pip install --target=$nb_path spacy

In [6]:
from ezc3d import c3d

ModuleNotFoundError: ignored

### Create pandas df
#### RUNNING

In [ ]:
fname_c3d_R = os.path.join(pathname,'RBDS001runT35.c3d')
c_R = c3d(fname_c3d_R)

In [ ]:
point_data_R = c_R['data']['points']
points_residuals_R = c_R['data']['meta_points']['residuals']
analog_data_R = c_R['data']['analogs']

### Rename marker labels to be consistent with RIC

In [ ]:
mkr_R_labels_RBDS = c_R['parameters']['POINT']['LABELS']['value']

In [ ]:
print(mkr_R_labels_RBDS)

In [ ]:
mkr_R_labels_RBDSm=['LASI', 'LHED', 'LHEL', 'LHEP', 'LIC', 'LMH1', 'LMH5', 
                    'LPSI', 'LSBL', 'LSBM', 'LSTL', 'LSTM', 
                    'LTBL', 'LTBM', 'LTTL', 'LTTM', 'RASI', 
                    'RHED', 'RHEL', 'RHEP', 'RIC', 'RMH1', 'RMH5', 'RPSI', 
                    'RSBL', 'RSBM', 'RSTL', 'RSTM', 
                    'RTBL', 'RTBM', 'RTTL', 'RTTM']

In [ ]:
mkr_R_data_RBDS = np.empty(shape=(point_data_R.shape[2],3*len(mkr_R_labels_RBDSm)))
for m, marker in enumerate(mkr_R_labels_RBDSm):
    mkr_R_data_RBDS[:,3*m:3*m+3] = point_data_R[:3, m, :].T

In [ ]:
xyz = list('XYZ')*len(mkr_R_labels_RBDSm)
mkr_R_labels_RBDS_2 = [ele for ele in mkr_R_labels_RBDSm for i in range(3)]
mkr_R_labels_RBDS_3 = [mkr_R_labels_RBDS_2[i]+'_'+xyz[i] for i in range(len(xyz))]

In [ ]:
df_R_RBDS = pd.DataFrame(data=mkr_R_data_RBDS, columns=mkr_R_labels_RBDS_3)

In [ ]:
duration_R  = point_data_R.shape[2]/int(c_R['parameters']['POINT']['RATE']['value'][0])
timeStamps_R= np.linspace(0, duration_R, mkr_R_data_RBDS.shape[0])

In [ ]:
df_R_RBDS['Time'] = timeStamps_R
df_R_RBDS.set_index('Time', inplace=True)
df_R_RBDS.index.name = 'Time'

In [ ]:
df_R_RBDS.head()

### STATIC

In [ ]:
fname_c3d = os.path.join(pathname,'RBDS001static.c3d')
c = c3d(fname_c3d)

In [ ]:
point_data = c['data']['points']
points_residuals = c['data']['meta_points']['residuals']
analog_data = c['data']['analogs']

In [ ]:
mkr_S_labels_RBDS = c['parameters']['POINT']['LABELS']['value']
print(mkr_S_labels_RBDS)

In [ ]:
mkr_S_labels_RBDSm=['LMAL', 'LMAM', 'LASI', 'LGTR', 'LHED', 'LHEL', 'LHEP', 
                    'LFIB', 'LIC', 'LKNL', 'LKNM', 'LMH1', 'LTOE', 'LMH5', 'LPSI', 'LSBL', 
                    'LSBM', 'LSTL', 'LSTM', 'LTBL', 'LTBM', 
                    'LTTL', 'LTTM', 'LTUB', 'RMAL', 'RMAM', 'RASI', 'RGTR', 'RHED', 
                    'RHEL', 'RHEP', 'RFIB', 'RIC', 'RKNL', 'RKNM', 'RMH1', 'RTOE', 'RMH5', 'RPSI', 
                    'RSBL', 'RSBM', 'RSTL', 'RSTM', 'RTBL', 
                    'RTBM', 'RTTL', 'RTTM', 'RTUB']

In [ ]:
mkr_S_data_RBDS = np.empty(shape=(point_data.shape[2],3*len(mkr_S_labels_RBDSm)))
for m, marker in enumerate(mkr_S_labels_RBDSm):
    mkr_S_data_RBDS[:,3*m:3*m+3] = point_data[:3, m, :].T

In [ ]:
xyz = list('XYZ')*len(mkr_S_labels_RBDSm)
mkr_S_labels_RBDS_2 = [ele for ele in mkr_S_labels_RBDSm for i in range(3)]
mkr_S_labels_RBDS_3 = [mkr_S_labels_RBDS_2[i]+'_'+xyz[i] for i in range(len(xyz))]

In [ ]:
df_S_RBDS = pd.DataFrame(data=mkr_S_data_RBDS, columns=mkr_S_labels_RBDS_3)

In [ ]:
duration  = int(c['parameters']['POINT']['RATE']['value'][0])/point_data.shape[2]
timeStamps= np.linspace(0, duration, mkr_S_data_RBDS.shape[0])

In [ ]:
df_S_RBDS['Time'] = timeStamps
df_S_RBDS.set_index('Time', inplace=True)
df_S_RBDS.index.name = 'Time'

In [ ]:
df_S_RBDS.head()

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

# Data for three-dimensional scattered points
zdata = df_S.values[100,2::3]
xdata = df_S.values[100,0::3]
ydata = df_S.values[100,1::3]
ax.scatter3D(xdata, ydata, zdata, c='b');
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_aspect('equal')
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

# Data for three-dimensional scattered points
zdata = df_S_RBDS.values[100,2::3]
xdata = df_S_RBDS.values[100,0::3]
ydata = df_S_RBDS.values[100,1::3]
ax.scatter3D(xdata, ydata, zdata, c='r');
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_aspect('equal')
plt.show()

### Save df to ascii
RBDS and RIC

# Static
fname_out_RBDS = os.path.join(pathname, 'RBDS_static.csv')
df_S_RBDS.to_csv(fname_out_RBDS)
# Running
fname_out_R_RBDS = os.path.join(pathname, 'RBDS_run.csv')
df_R_RBDS.to_csv(fname_out_R_RBDS)

# Static
fname_out_RIC = os.path.join(pathname, 'RIC_static.csv')
df_Sm.to_csv(fname_out_RIC)
# Running
fname_out_R_RIC = os.path.join(pathname, 'RIC_run.csv')
df_R_RICm.to_csv(fname_out_R_RIC)

In [ ]:
fname_c3d_f = os.path.join(pathname,'RBDS001runT35.c3d')
c3d = c3d(fname_c3d_f, extract_forceplat_data=True);

pf_0 = c3d["data"]["platform"][0]  # Select the first platform

In [ ]:
pf_0['unit_force']          # Units of forces
pf_0['unit_moment']         # Units of moments
pf_0['unit_position']       # Units of center of pressure

pf_0['cal_matrix']          # Calibration matrix
pf_0['corners']             # Position of the corners
pf_0['origin']              # Position of the origin

pf_0['force']               # Force data
pf_0['moment']              # Moment data
pf_0['center_of_pressure']  # Center of pressure data
pf_0['Tz']                  # Moment at center of pressure data

In [ ]:
pf_0['cal_matrix'] 

In [ ]:
    if df_multi:  # dataframe with multiple labels
        df.drop(labels='Frame#', axis=1, inplace=True)
        df.set_index('Time', inplace=True)
        df.index.name = 'Time'
        cols = [s[:-1] for s in df.columns.str.replace(r'.', r'_')]
        df.columns = [cols, list('XYZ')*int(df.shape[1]/3), xyz]
        df.columns.set_names(names=['Marker', 'Coordinate', 'XYZ'],
                             level=[0, 1, 2], inplace=True)

In [ ]:
mkr_S_data.shape

In [ ]:
df_s = pd.DataFrame.from_dict(data['out']['full_static'])
df_s.head()

In [ ]:
data['out']['full_static']

In [ ]:
print(data['out'].keys())

In [ ]:
data['out']['neutral'].keys()

In [ ]:
data['out']['full_static'].keys()

In [ ]:
data['out'][0][0]

In [ ]:
data['__header__']

### Data stored in RBA directory

In [ ]:
fname_q= os.path.join(pathname, 'RBDS_google_forms.csv') 
# Import data
data_q = pd.read_csv(fname_q, sep = ',', index_col = 'ID')
data_q.head()

In [ ]:
df_rba = data_q[['Altura (cm)','Massa (kg)']]
df_rba

In [ ]:
df_figshare = info[['Subject','Height','Mass']]
df_figshare.set_index('Subject')

In [ ]:
df_out = df_rba.loc[df_rba['Altura (cm)']==df_figshare['Height'].values[5]]
df_out.index.tolist()

In [ ]:
ids = []
for i in range(df_figshare.shape[0]):
    df_out = df_rba.loc[(df_rba['Altura (cm)']==df_figshare['Height'].values[i]) & (df_rba['Massa (kg)']==df_figshare['Mass'].values[i])]
    if df_out.empty:
        print('Subject '+ str(df_figshare['Subject'].values[i]) + ' not found!')
    else:
        ids.append(df_out.index.tolist())

In [ ]:
ids

# PENDING
* Find who is who. 
Find the subjects in Figshare RBDSinfo.txt with the corresponding ID in RBA spreadsheet